In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader as dr
from mpl_toolkits.mplot3d import axes3d
from scipy.stats import norm
from scipy.interpolate import interp2d
import math as m

### European Call option value $\mathbf{E}[(F(T) - e^{-rT}E)^+]$

In [ ]:
# Parameters
M = 10**4 # No. of Monte Carlo Simulations
F_0 = 2500 # F(0)
alpha = 0.01 # alpha coefficient in the SABR model (was 1 for Aug20 results)
rho = 0.25 # correlation between F(t) and sigma(t), rho is an element of [-1, 1]
# theta = 0.5
sig_0 = 0.1 # sigma(0)

In [ ]:
# Function to calculate F using the implicit EM approximation of the Lamperti transforamtion
def Implicit(T, N, sig_0, M, alpha, F_0, rho, theta):
    Y_0 = (1/(1-theta))*F_0**(1-theta)
    dt = T/N # time step
    Y_n = np.ones(M)*Y_0 # array to hold values for Y at t for each monte carlo simulation
    sig_t = np.ones(M)*sig_0 # array to hold values for sigma at t for each monte carlo simulation
    A = np.array([alpha*rho, alpha*np.sqrt(1 - rho**2)], dtype=float) # Cholesky factor
    for i in range(int(N)):
        B = np.sqrt(dt) * np.random.normal(0, 1, (2, M)) # Brownian motion
        B2 = A[0]*B[0,:] + A[1]*B[1,:] # Brownian motion for B2 in the SABR model
        sig_t = sig_t*np.exp((-alpha**2 / 2)*dt + B2) # sigma(t)
        B1 = sig_t*B[0,:] # Brownian motion for B1 in the SABR model
        Y_n = ((Y_n + B1)/2) + np.sqrt((((Y_n + B1)**2)/4) - ((sig_t**2) * dt * theta) / (2 * (1 - theta)))
        
    F_n = (Y_n * (1 - theta))**(1/(1 - theta)) # convet Yn to Fn
    return np.mean(F_n), np.mean(sig_t) # return expected value of F(T)

In [ ]:
# Function to calculate option value
def OptionValue(F, E, T, r):
    return np.maximum(F - np.exp(-r*T)*E, 0)

In [ ]:
# Black-Scholes formula
def BSF(S, E, T, sigma, r):
    # where:
    # S is the initial value of the underlying asset
    # E is the stike price
    # r is the risk free rate
    # sigma is the volatility of the underlying asset
    # T is the time to maturity
    # D is the dividend yield
    # option is to define whether a call or put option is being valued
    
    # d1 is a variable in the black-scholes formula
    d1 = (np.log(S / E) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    # d2 is a variable in the black-scholes formula
    d2 = (np.log(S / E) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    value = S * 0.5 * norm.cdf(d1) - E * np.exp(-r * T) * 0.5 * norm.cdf(d2)
    return value

In [ ]:
# d1 variable from the Black-Scholes formula
def Vega(S, E, T, sigma, r):
    
    # d1 is a variable in the black-scholes formula
    d1 = (np.log(S / E) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    
    return vega

In [ ]:
# Uncomment if you want to use the US treasury and Libor rate
# syms = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3']
# yc = dr(syms, 'fred') # could specify start date with start param here
# names = dict(zip(syms, ['1m', '3m', '6m', '1yr', '2yr']))
# yc = yc.rename(columns=names)
# yc = yc[['1m', '3m', '6m','1yr', '2yr']]
# USr = yc.iloc[-1,:].to_numpy()/100 # US treasury constant maturity rate
# # libor rates effective from the 28/05/2021
# rWeek = dr('USD1WKD156N', 'fred').iloc[-1]/100 # 1 week libor rate

time = np.array([1/52, 1/12, 1/4, 1/2, 1, 2]) # time i.e. 1 week = 1/52, 1 month = 1/12
# r = np.insert(USr, 0, rWeek)
r = [0.005, 0.01, 0.02, 0.025, 0.03, 0.05] # term structure

In [ ]:
T = np.arange(time[0], time[-1]+1/52, 1/52) # time array that goes from 1 week to 2 years in 1 week increments
rInterp = np.interp(T, time, r) # interpolate term structure for time array, T
N = np.round(T/(1/1460),0) # Array of number of steps, this is to maintain a dt=1day 

In [ ]:
# Bisection method for calculating implied volatility
def bisection(S, E, T, r, tol, MP, sigma):
    # MP is options market price
    upper_vol = 4 # upper estimate
    lower_vol = 0.005 # lower estimate
    iteration = 0 # iteration counter
    tol = 1e-8 # degree of accuracy
    eps = 1 # placeholder for error between estimates 
    max_iter = 1000 # max iteration limit (avoid infinte loop)
    BS_Low = BSF(S=S, E=E, T=T, sigma=lower_vol, r=r) # BS price for low estimate
    BS_High = BSF(S=S, E=E, T=T, sigma=upper_vol, r=r) # BS price for high estimat
    vi = lower_vol + (MP - BS_Low)*(upper_vol - lower_vol)/(BS_High - BS_Low) # Bisection formula
    tempVal = BSF(S=S, E=E, T=T, sigma=vi, r=r) # BS price for latest estimate
    while eps > tol:
        iteration += 1
        if iteration >= max_iter:     # the count will be used to break an infinte loop from occuring
            print('break E={}, T={}, MP={}, S={}, E+MP-S={}'.format(E, T, MP, S, E+MP-S))
            break
        if tempVal < MP:
            lower_vol = vi
        else:
            upper_vol = vi
            
        BS_Low = BSF(S=S, E=E, T=T, sigma=lower_vol, r=r)
        BS_High = BSF(S=S, E=E, T=T, sigma=upper_vol, r=r)
        vi = lower_vol + (MP - BS_Low)*(upper_vol - lower_vol)/(BS_High - BS_Low)
        tempVal = BSF(S=S, E=E, T=T, sigma=vi, r=r)
        eps = abs(MP - tempVal)
    return vi

In [ ]:
# Newton-Raphson method for calculating implied volatility (IV)
def Newton(MP, S1, E1, r1, T1):
    epsilon = 1 # constant to be used in the calculation
    tol = 1e-5 # tolerance of volatility estimate
    count = 0 # count how many estimates are required 
    max_iter = 10000 # cut the loop once max_iter has been reached
    sigma = np.sqrt(np.abs(np.log(S1/E1) + r1*T1)*(2/T1)) # initial estimate of volatility
    while epsilon > tol:
        count += 1                # Incase there is an error in the calculation, 
        if count >= max_iter:     # the count will be used to break an infinte loop from occuring
            print('break E={}, T={}, MP={}, S={}, E+MP-S={}'.format(E1, T1, MP, S1, E1+MP-S1))
            break
        orig_vol = sigma # store the previous estimate
        BSP = BSF(S=S1, E=E1, T=T1, sigma=sigma, r=r1) # Black-Scholes price from previous estimate
        vega = Vega(S=S1, E=E1, T=T1, sigma=sigma, r=r1)
        if vega == 0:
            sigma = bisection(S=S1, E=E1, T=T1, r=r1, MP=MP, sigma=sigma)
            return sigma
        sigma = sigma - (BSP - MP) / vega # Newton-Raphson calculation
        epsilon = abs(sigma - orig_vol) # Tolerance to be obtained in the estimate to break the calultion loop
    if m.isnan(sigma)==True or sigma < 0:
        sigma = bisection(S=S1, E=E1, T=T1, r=r1, MP=MP, sigma=sigma)
    return sigma

In [ ]:
# Generate an array of Strike prices, this range from 0.8*F(0) to 1.1*F(0)
def StrikePrice(F_0):
    return np.linspace(F_0*0.8, F_0*1.1, len(T)-1)

In [ ]:
# Calculate implied volatility of options
def ImpliedVolatility(T, F_0, r, alpha, rho, theta, sig_0, N, M, E):
    F_T, sigma = Implicit(T=T, N=N, sig_0=sig_0, M=M, alpha=alpha, F_0=F_0, rho=rho, theta=theta) # Simulate Forward prices
    V = OptionValue(F=F_T, E=E, T=T, r=r) # calculate option values
    IV = Newton(MP=V, S1=F_0, E1=E, r1=r, T1=T) # produce implied volatilities
    return IV, V, F_T, sigma

In [ ]:
theta = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) # evaluate over range of theta values
ImpVol = np.zeros((len(theta), len(T)-1, len(T))) # matrix to store implied volatility
OptionPrice = np.zeros((len(theta), len(T)-1, len(T))) # matrix to store option price values
F = np.zeros((len(theta), len(T)-1, len(T))) # matrix to store forward prices
sig = np.zeros((len(theta), len(T)-1, len(T))) # matrix to store sigma from forward prices
E = StrikePrice(F_0=F_0)
for k in range(len(theta)):
    for j in range(len(T)):
        for i in range(len(E)):
            ImpVol[k, i, j], OptionPrice[k, i, j], F[k, i, j], sig[k, i, j] = ImpliedVolatility(T=T[j], F_0=F_0, r=rInterp[j], alpha=alpha, 
                                             rho=rho, theta=theta[k], sig_0=sig_0, N=N[j], M=M, E=E[i])

In [ ]:
# plot the volatility surface
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection='3d')
X, Y = np.meshgrid(T, E)
s = ax.plot_surface(X, Y, ImpVol[5,:,:], cmap = 'Spectral')
ax.set_xlabel('Time to Maturity', fontsize=14)
ax.set_ylabel('Strike', fontsize=14)
ax.set_zlabel('Volatility', fontsize=14)
ax.set_title('Implied Volatility Surface', fontsize=16)
fig.colorbar(s, shrink=0.5, aspect=20)
plt.show()

In [ ]:
# Plot volatility smile
plt.plot(E, npVol[:,12])
plt.xlabel('Stike Price')
plt.ylabel('Implied Volatility')
plt.show()